In [34]:
import xarray as xr
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np
import os
import pulp
import matplotlib.pyplot as plt
import tqdm
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime
import pytz
import netCDF4 as nc
import tqdm
from calendar import monthrange

#create dummy expansion file with one row years from 2021 to 2030 and the other column expansion starting from -14000000 in 2021 and going up in 7000000 steps
# Create a DataFrame from 2023 to 2050 with each year increasing by 10000 in expansion
data = {
    "Year": [2020,2021,2022,2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075],
    "PV Expansion": [0.0,0.0,0.0,0.0, 13.9, 20.0, 20.0, 22.0, 22.0, 21.5, 21.5, 18.8, 18.8, 18.8, 18.8, 18.8, 18.2, 18.2, 18.2, 18.2, 18.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
}
expansion_df = pd.DataFrame({
    "Year": data["Year"],
    "PV Expansion": data["PV Expansion"]
})
expansion_df['PV Expansion'] = expansion_df['PV Expansion']*1000000 #convert to kW
plz_shapefile=gpd.read_file('georef-germany-postleitzahl.shp')
plz_shapefile=plz_shapefile.to_crs('EPSG:4326')
#make plz_shapefile a df
plz_df=pd.DataFrame(plz_shapefile)

# sdr=xr.open_dataset('get_data/PV-Input/rsds_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_SMHI-RCA4_v1a_mon_202101-203012.nc') 
# ws=xr.open_dataset('get_data/PV-Input/sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_SMHI-RCA4_v1a_mon_202101-203012.nc')
# #TODO: Maxtemp
# at=xr.open_dataset('get_data/PV-Input/tasmax_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_SMHI-RCA4_v1a_mon_202101-203012.nc')


In [4]:
def process_shapefiles_with_PV_data_optimized(plz_shapefile, sdr, varname):
           # Read the PLZ shapefile
        plz_gdf = plz_shapefile.copy()
        #adjust the crs to a metric crs as this will help the buffer
        sdr.crs = 'epsg:4647'

        plz_gdf.crs='epsg:4647'

        

        # Define time columns and initialize new columns in plz_gdf
        time_cols = [col for col in sdr.columns if col not in ['geometry']]
        for time_col in time_cols:
            plz_gdf[f'{varname}{time_col}'] = None
        plz_gdf['point_ids'] = None
        plz_gdf['point_count'] = 0
        plz_gdf['type'] = None
        plz_gdf['closest_point_distance'] = None

        # Spatial join for points within polygons
        joined_gdf = gpd.sjoin(plz_gdf, sdr, how='left', op='contains')
        for idx, group in joined_gdf.groupby(joined_gdf.index):
            data_pts = group.dropna(subset=['ID'])
            plz_gdf.at[idx, 'point_ids'] = list(data_pts['ID'].astype(int))
            plz_gdf.at[idx, 'point_count'] = len(data_pts)
            plz_gdf.at[idx, 'type'] = 'within' if len(data_pts) > 0 else None
            for time_col in time_cols:
                plz_gdf.at[idx, f'{varname}{time_col}'] = data_pts[time_col].mean()

        # Optimized handling of PLZ polygons without wind points
        no_value_plz = plz_gdf[plz_gdf['point_count'] == 0]
        buffer_distance = 14000  # 14 km buffer
        sindex = sdr.sindex  # Spatial index for sdr

        for idx, row in no_value_plz.iterrows():
            # Use spatial index to narrow down the candidates
            potential_matches_index = list(sindex.intersection(row.geometry.buffer(buffer_distance).bounds))
            potential_matches = sdr.iloc[potential_matches_index]
            # Calculate the nearest point
            nearest_point_data = potential_matches.distance(row.geometry).idxmin()
            nearest_point = sdr.loc[nearest_point_data] if pd.notna(nearest_point_data) else None

            if nearest_point is not None:
                nearest_point_id = nearest_point.name
                plz_gdf.at[idx, 'point_ids'] = [nearest_point_id]
                plz_gdf.at[idx, 'point_count'] = 1
                plz_gdf.at[idx, 'type'] = 'nearby'
                plz_gdf.at[idx, 'closest_point_distance'] = row.geometry.distance(nearest_point.geometry)
                for time_col in time_cols:
                   plz_gdf.loc[idx, f"{varname}{time_col}"] = nearest_point[time_col]
        return plz_gdf

In [5]:
def create_germany_gdf_from_ds(ds, varname, lat_min=46, lat_max=56, lon_min=5, lon_max=16):
    """
    Creates a GeoDataFrame containing points within Germany's geographic bounds from a given dataset.
    
    Parameters:
    - ds: The dataset containing latitude, longitude, and wind speed data.
    - lat_min, lat_max, lon_min, lon_max: Geographic bounds for filtering the points.
    
    Returns:
    - A GeoDataFrame with points within the specified bounds and wind speed data for each time step.
    """
    # Create a DataFrame from dataset coordinates
    lat_lon_df = pd.DataFrame({
        'lat': ds['lat'].values.ravel(),
        'lon': ds['lon'].values.ravel()
    })

    # Create a GeoDataFrame from lat_lon_df
    gdf = gpd.GeoDataFrame(lat_lon_df, geometry=gpd.points_from_xy(lat_lon_df.lon, lat_lon_df.lat))

    # Add each time step as a new column with simplified name 'year-month'
    for time in ds['time'].values:
        simplified_time_name = pd.to_datetime(time).strftime('%Y-%m')
        data_slice = ds.sel(time=time)
        gdf[simplified_time_name] = data_slice[varname].values.ravel()

    # Filter the GeoDataFrame for points within the specified geographic bounds
    germany_gdf = gdf[(gdf['lat'] >= lat_min) & (gdf['lat'] <= lat_max) & (gdf['lon'] >= lon_min) & (gdf['lon'] <= lon_max)]
    germany_gdf['ID'] = germany_gdf.index
    #use only lat lon and geometry and the columns starting with year_oi
    #germany_gdf = germany_gdf[['lat', 'lon', 'geometry', 'ID'] + [col for col in germany_gdf.columns if col.startswith('20')]]
    #
    return germany_gdf

In [6]:

def process_year_for_germany(germany_gdf_template, varname):
    results = []
    

    ds = pd.read_csv('Data/pv_interim/Germany_gdf_AT_RCP26.csv')
    
    # Create a base DataFrame from the coordinates (if not already created)
    if germany_gdf_template is None:
        lat_lon_df = pd.DataFrame({
            'lat': ds['lat'].values.ravel(),
            'lon': ds['lon'].values.ravel()
        })
        germany_gdf = gpd.GeoDataFrame(lat_lon_df, geometry=gpd.points_from_xy(lat_lon_df.lon, lat_lon_df.lat))
    else:
        germany_gdf = germany_gdf_template.copy()
    
    # Extract all years data for 'sfcWind'
    for time in ds['time'].values:
        simplified_time_name = pd.to_datetime(time).strftime('%Y-%m')
        data_slice = ds.sel(time=time)
        germany_gdf[simplified_time_name] = data_slice[varname].values.ravel()
    
    # Add dataset identifier based on the path (to distinguish between RCP scenarios)
    rcp_id = path.split('_')[3]  # Assumes RCP info is the 4th element in the filename
    germany_gdf['RCP'] = rcp_id
    
    results.append(germany_gdf)
    
    return results

In [7]:
def generate_id_to_plz_mapping(germany_gdf):
    # This assumes you have a function to get the nearest ID as previously discussed
    # Assuming process_shapefiles_with_PV_data_optimized returns a DataFrame with 'nearest_ID'
    #plz_gdf = process_shapefiles_with_PV_data_optimized(plz_shapefile, germany_gdf, varname="your_varname")

    # Create a dictionary to map ID to PLZ and LAN codes
    mapping_dict = germany_gdf.drop_duplicates(subset='plz_code').set_index('plz_code')[['lan_code', 'ID']].to_dict('index')

    return mapping_dict

In [8]:
# Merge the mapping DataFrame with the detailed data DataFrame
# Convert dictionary to DataFrame for easy merging
def map_data_to_plz(germany_gdf, mapping_dict, varname, year_oi):
    # Create DataFrame from mapping dictionary and rename 'index' to 'plz_code'
    plz_mapping_df = pd.DataFrame.from_dict(mapping_dict, orient='index').reset_index().rename(columns={'index': 'plz_code'})

    #make ID col string
    plz_mapping_df['ID'] = plz_mapping_df['ID'].astype(int)
    germany_gdf['ID'] = germany_gdf['ID'].astype(int)
    # Merge with germany_gdf to get additional data based on ID
    result_df = plz_mapping_df.merge(germany_gdf, on='ID', how='left')

    # Save the resulting DataFrame to a CSV file
    result_df.to_csv(f'Data/pv_interim/Germany_gdf_{varname}_{year_oi}.csv', index=False)

    return result_df

In [9]:
def apply_mappings(germany_gdf, id_to_plz_mapping):
    # Assume germany_gdf has an 'ID' column which matches keys in id_to_plz_mapping
    germany_gdf['plz_code'] = germany_gdf['ID'].apply(lambda x: id_to_plz_mapping[x]['plz_code'] if x in id_to_plz_mapping else None)
    germany_gdf['lan_code'] = germany_gdf['ID'].apply(lambda x: id_to_plz_mapping[x]['lan_code'] if x in id_to_plz_mapping else None)

    return germany_gdf


In [10]:
def process_year(germany_gdf,plz_shapefile,varname,scenario):

    

    
    #####für test ds: nur die ersten 100 zeilen von plz
    #plz=plz.head(100)



    germany_gdf = germany_gdf[['ID','lat', 'lon','geometry'] + [col for col in germany_gdf.columns if col.startswith('20')]]
    germany_gdf = gpd.GeoDataFrame(germany_gdf, geometry='geometry')


    # %%
    updated_plz_gdf = process_shapefiles_with_PV_data_optimized(plz, germany_gdf,varname)

    print("updated_plz_gdf erzeugt")

    # %%
    #drop unnecessary columns name, plz_name, plz_name_lo, krs_code, lan_name, lan_code, krs_name, plz_int
    #updated_plz_gdf.drop(['name', 'plz_name', 'plz_name_lo', 'krs_code', 'lan_name', 'krs_name', 'PLZ_int'], axis=1, inplace=True)
    #updated_plz_gdf.drop(['point_count','point_ids','closest_point_distance','type'], axis=1, inplace=True)
    #check columns
    updated_plz_gdf.columns

    # %%

    #check if values double in plz_code column
    updated_plz_gdf['plz_code'].value_counts()
    #delete duplicates
    updated_plz_gdf.drop_duplicates(subset='plz_code', keep='first', inplace=True)
    
    #delete length of varname from column names if startswith varname
    updated_plz_gdf.columns = [col[len(varname):] if col.startswith(varname) else col for col in updated_plz_gdf.columns]
    #save as csv
    updated_plz_gdf.to_csv(f'Data/pv_interim/Germany_gdf_{varname}_{scenario}.csv')
    return updated_plz_gdf

In [32]:

def kelvin_to_celsius(germany_gdfAT, varname, year_oi, scenario):
    year_oistr=str(year_oi)
    scenario_str=str(scenario)  
    #convert germany_gdf to °C
    cols_to_convert = [col for col in germany_gdfAT if col.startswith('20')]

    # Apply the conversion formula to each of these columns
    for col in cols_to_convert:
        germany_gdfAT[col] = germany_gdfAT[col] - 273.15
    germany_gdfAT.to_csv(f'Data/pv_interim/Germany_gdf_{varname}_{scenario_str}_{year_oistr}.csv')
    return germany_gdfAT
    

In [96]:

def avg_daily_sunhours(year_oi,scenario):
    year_oistr=str(year_oi)
    germany_gdfSDR = pd.read_csv(f'Data/pv_interim/{scenario}/germany_gdf_SDR_{scenario}_{year_oi}.csv')
    timezone = 'Europe/Berlin'
    #create empty dataframe to store the results with the columns plz_code, and 01-2024, 02-2024, 03-2024, 04-2024, 05-2024, 06-2024, 07-2024, 08-2024, 09-2024, 10-2024, 11-2024, 12-2024
    daylight_df=pd.DataFrame(columns=['plz_code']+['lat']+['lon'] +[year_oistr+'-'+f'{str(i).zfill(2)}' for i in range(1,13)])  


    daylight_df['plz_code']=germany_gdfSDR['plz_code']
    daylight_df['lat']=germany_gdfSDR['lat']
    daylight_df['lon']=germany_gdfSDR['lon']


    #delete rows with nan values
    daylight_df.dropna(inplace=True)
    # Assuming 'daylight_df' is already initialized and contains 'plz_code', 'lat', 'lon', and month columns

    for index, row in daylight_df.iterrows():
        # Extract latitude and longitude for the current row
        lat, lon = row['lat'], row['lon']
        
        for month_col in [year_oistr+'-'+f'{str(i).zfill(2)}' for i in range(1, 13)]:  # Month columns
            # Parse the month and year from the column name
            month = int(month_col.split('-')[1])
            year = int(month_col.split('-')[0])
            
            # Set the date to the 15th of each month in year_oi
            date = datetime.datetime(year, month, 15)
            date = date.replace(tzinfo=pytz.timezone(timezone))
            
            # Create a LocationInfo object for the location
            loc = LocationInfo(latitude=lat, longitude=lon)
            
            # Calculate solar events (sunrise, sunset) for the date
            solar_events = sun(loc.observer, date=date)
            
            # Calculate the duration of daylight in hours
            daylight_duration = (solar_events['sunset'] - solar_events['sunrise']).total_seconds() / 3600
            
            # Update the DataFrame with the calculated daylight hours
            daylight_df.at[index, month_col] = daylight_duration

    #save as csv
    daylight_df.to_csv(f'Data/pv_interim/daylight_df_{scenario}_{year_oistr}.csv')
    # Assuming you might want to see the DataFrame or use it further
    return daylight_df

In [15]:
# import calendar

# # Iterate over the columns (months) in daylight_df to calculate total daylight hours
# for month_col in [year_oistr+'-'f'{str(i).zfill(2)}' for i in range(1, 13)]:
#     # Extract the month number from the column name
#     month = int(month_col.split('-')[1])
    
#     # Get the number of days in the month
#     days_in_month = calendar.monthrange(year_oi, month)[1]
    
#     # Multiply the daylight hours for the 15th by the number of days in the month
#     # Update the DataFrame with the total daylight hours for each month and location
#     daylight_df_cum=daylight_df.copy()
#    # daylight_df_cum[[month_col]] = daylight_df[month_col] * days_in_month

In [16]:
def adjust_sdr_rates(germany_gdfSDR, daylight_df, year_oi):
    """
    # Since the values of germany_gdfSDR are the monthly average SDR in W/m^2  we have to firstly readjust the values to the daytime radiation, since at night it's virtually zero.
    # This can be done by multiplying the monthly average by the daylight hours of the respective month divided by 24 hours.
    # This will give us the average radiation for the daylight hours of the respective month.
    # Next we will adjust the W/m² values to kWh/m² by multiplying the values by the daylight hours (to have Wh) and then by 0.001 to convert them to kWh/m².
    # Iterate through each month column in germany_gdfSDR
    Adjust SDR rates based on actual daylight hours and convert to kWh/m².
    
    Parameters:
    - germany_gdfSDR: DataFrame containing SDR rates and location data (plz_code, lat, lon).
    - daylight_df: DataFrame containing total daylight hours for each month.
    - year_oi: The year of interest.
    
    Returns:
    - adj_SDR_rate: DataFrame with adjusted SDR rates in kWh/m² and location data.
    """
    year_oistr = str(year_oi)
    # Generate days in each month for year_oi
    days_in_month = {f"{year_oistr}-{str(i).zfill(2)}": calendar.monthrange(year_oi, i)[1] for i in range(1, 13)}
    
    # Create empty dataframe to store the results
    adj_SDR_rate = pd.DataFrame(columns=['plz_code', 'lat', 'lon'] + [f"{year_oistr}-{str(i).zfill(2)}" for i in range(1,13)])
    
    # Initialize adj_SDR_rate DataFrame with the structure and data from germany_gdfSDR
    adj_SDR_rate['plz_code'] = germany_gdfSDR['plz_code']
    adj_SDR_rate['lat'] = germany_gdfSDR['lat']
    adj_SDR_rate['lon'] = germany_gdfSDR['lon']
    
    for month_col in [f"{year_oistr}-{str(i).zfill(2)}" for i in range(1, 13)]:
        if month_col in germany_gdfSDR.columns and month_col in daylight_df.columns:
            # Adjust SDR rates for actual daylight hours and convert to kWh/m²
            # Note: Assuming daylight_df contains total daylight hours for each month
            adjusted_sdr = germany_gdfSDR[month_col] * (24 / daylight_df[month_col])
            
            # Store the adjusted SDR rates in adj_SDR_rate DataFrame
            adj_SDR_rate[month_col] = adjusted_sdr
            
            # Convert adjusted SDR from W/m² to kWh/m² for the month
            # Assuming the formula should actually apply to adj_SDR_rate instead of modifying germany_gdfSDR again
            adj_SDR_rate[month_col] = adjusted_sdr * daylight_df[month_col] * days_in_month[month_col] * 0.001
    
    return adj_SDR_rate



In [78]:
import pandas as pd
import numpy as np

def calculate_Tpv(year_oi,scenario):
    """
    Calculates Tpv values for each month based on input DataFrames.

    Parameters:
    - year_oi: Year of interest as a string, e.g., '2024'.
    - germany_gdfAT: DataFrame containing AT data with 'plz_code' and monthly values.
    - germany_gdfSDR: DataFrame containing SDR data with 'plz_code' and monthly values.
    - germany_gdfWS: DataFrame containing WS data with 'plz_code' and monthly values.

    Returns:
    - Tpv_df: DataFrame containing the calculated Tpv values along with 'plz_code', 'lat', 'lon' for each month.
    """
    year_oistr = str(year_oi)   
    germany_gdfAT=pd.read_csv(f'Data/pv_interim/{scenario}/germany_gdf_AT_{scenario}_{year_oi}.csv')
    germany_gdfSDR=pd.read_csv(f'Data/pv_interim/{scenario}/germany_gdf_SDR_{scenario}_{year_oi}.csv')
    germany_gdfWS=pd.read_csv(f'Data/pv_interim/{scenario}/germany_gdf_WS_{scenario}_{year_oi}.csv')
    germany_gdfAT=kelvin_to_celsius(germany_gdfAT, 'AT', year_oi, scenario)
    # Create empty DataFrame to store the results
    Tpv_df = pd.DataFrame(columns=['plz_code', 'lat', 'lon'] + [f'{year_oi}-{str(i).zfill(2)}' for i in range(1, 13)])
    Tpv_df['plz_code'] = germany_gdfSDR['plz_code']
    Tpv_df['lat'] = germany_gdfSDR['lat']
    Tpv_df['lon'] = germany_gdfSDR['lon']
    
    # Identify all month columns by excluding non-month columns
    month_columns = [col for col in germany_gdfAT.columns if col.startswith(year_oistr)]
    
    # Initialize Tpv DataFrame with plz_code and NaN for month columns
    Tpv = pd.DataFrame({'plz_code': germany_gdfAT['plz_code']})
    for month in month_columns:
        Tpv[month] = np.nan  # Initialize with NaN
    
    # Calculate Tpv for each month using the provided formula
    for month in month_columns:
        Tpv[month] = 2.08 + 1.038 * germany_gdfAT[month] + 0.0182 * germany_gdfSDR[month] - 1.13 * germany_gdfWS[month]
    
    #save as csv
    Tpv.to_csv(f'Data/pv_interim/Tpv_df_{scenario}_{year_oi}.csv',sep=';')
    return Tpv
# year_oi = '2024'
# Tpv_df = calculate_Tpv(year_oi, germany_gdfAT, germany_gdfSDR, germany_gdfWS)
# Make sure to replace `germany_gdfAT`, `germany_gdfSDR`, and `germany_gdfWS` with the actual DataFrames containing your data.



In [25]:
def calculate_annual_contraction(start_year, end_year,typ, input_dir='', output_dir='Data/pv_interim/'):
    # Load the combined MASTR data
    file_path = f'2023MASTR{typ}.csv'
    mastr_data = pd.read_csv(file_path, sep=';')
    mastr_data['Inbetriebnahmedatum'] = pd.to_datetime(mastr_data['Inbetriebnahmedatum'])

    # Loop through each year to calculate decommissioning for exactly 20-year-old turbines
    for year in range(start_year, end_year + 1):
        if year != start_year:
            # Filter installations that turn exactly 25 years old during the year
            filter_year = mastr_data['Inbetriebnahmedatum'].dt.year == (year - 25)
            yearly_decommissioned = mastr_data[filter_year]

            # Aggregate the decommissioned data by postcode
            aggregated_data = yearly_decommissioned.groupby('Postleitzahl').agg(
                Total_Nettonennleistung=('Nettonennleistung', 'sum'),
                Anzahl_Anlagen=('Postleitzahl', 'count'),

                Bundesland=('Bundesland', 'first')
            ).reset_index()

        else:
        # Filter installations that turn exactly 25 years old during the year
            filter_year = mastr_data['Inbetriebnahmedatum'].dt.year <= (year - 25)
            yearly_decommissioned = mastr_data[filter_year]

            # Aggregate the decommissioned data by postcode
            aggregated_data = yearly_decommissioned.groupby('Postleitzahl').agg(
                Total_Nettonennleistung=('Nettonennleistung', 'sum'),
                Anzahl_Anlagen=('Postleitzahl', 'count'),

                Bundesland=('Bundesland', 'first')
            ).reset_index()
        #rename Postleitzahl to plz_code
        aggregated_data.rename(columns={'Postleitzahl':'plz_code'}, inplace=True)
        
        # Save the yearly data
        aggregated_data.to_csv(f'{output_dir}/contraction_data_{year}_{typ}.csv', sep=';', index=False)
        print(f'Contraction data for year {year} saved, focusing on installations exactly 25 years old.')



In [40]:
def filter_active_installations(Typ, year_oi, expansion_df):
    """
    Filters active and deprecated PV installations based on their operational status and age.
    Updates the expansion target for the next year by adding the power from deprecated installations,
    unless it is the last year in the dataset.
    
    Parameters:
    - Typ: The type of installation as a string.
    - year_oi: The year of interest as an integer.
    - expansion_df: DataFrame containing expansion targets for years.
    
    Returns:
    - active_installations: DataFrame of installations not permanently decommissioned and age <= 25 years.
    - expansion_df: Updated DataFrame with adjusted expansion values for the next year.
    """
    year_befoistr = str(year_oi - 1)
    
    mastr_pv = pd.read_csv(f'{year_befoistr}MASTR{Typ}.csv', sep=';')
    
    # Pre-processing
    mastr_pv['Inbetriebnahmedatum'] = pd.to_datetime(mastr_pv['Inbetriebnahmedatum'], format='%Y-%m-%d')
    mastr_pv['Age_as_of_year_oi'] = year_oi - mastr_pv['Inbetriebnahmedatum'].dt.year
    
    # Filter active and deprecated installations
    active_installations = mastr_pv[(mastr_pv['EinheitBetriebsstatus'] != 'endgültig stillgelegt') & (mastr_pv['Age_as_of_year_oi'] <= 25)]
    deprecated_installations = mastr_pv[(mastr_pv['EinheitBetriebsstatus'] == 'endgültig stillgelegt') | (mastr_pv['Age_as_of_year_oi'] > 25)]
    
    # Calculate deprecated installations' power
    power_deprecated_installations = deprecated_installations['Nettonennleistung'].sum()
    
    # Check if the current year is the last year in the dataset
    if year_oi < 2050:
        next_year = year_oi + 1
        if next_year in expansion_df['Year'].values:
            expansion_df.loc[expansion_df['Year'] == next_year, 'PV Expansion'] += power_deprecated_installations

    # Print summary information about deprecated installations
    print(f"Number of deprecated installations in {year_oi}: {len(deprecated_installations)}")
    print(f"Power of deprecated installations in {year_oi}: {power_deprecated_installations} kWp")
    
    return active_installations, expansion_df


In [46]:
def aggregate_on_PLZ_level(Typ, year_oi, plz_df):
    
    """
    Aggregates the MaStR data on the PLZ level.
    
    Parameters:
    - mastr_pv: DataFrame containing the MaStR data.
    
    Returns:
    - A DataFrame with aggregated data on the PLZ level.
    """
    # Group by 'Postleitzahl' and aggregate the data
    year_befoistr = str(year_oi - 1)
    mastr_pv=pd.read_csv(f'{year_oi-1}MASTR{Typ}.csv', sep=';')
    # Pre-processing
    mastr_pv['Inbetriebnahmedatum'] = pd.to_datetime(mastr_pv['Inbetriebnahmedatum'], format='%Y-%m-%d')
    mastr_pv['Age_as_of_year_oi'] = year_oi - mastr_pv['Inbetriebnahmedatum'].dt.year

    PV_agg = mastr_pv.groupby('Postleitzahl').agg(
        BL=('Bundesland', 'first'),
        Total_BRUTTOLEISTUNG=('Nettonennleistung', 'sum'),
        num_installation=('Postleitzahl', 'count'),
        mean_age=('Age_as_of_year_oi', 'mean')
        
    ).reset_index()
    
    # Map the 'BL' column to 'BL_code' using the defined mapping
    bl_to_bl_code_mapping = {
        'Baden-Württemberg': '08', 'Bayern': '09', 'Berlin': '11', 'Brandenburg': '12', 
        'Bremen': '04', 'Hamburg': '02', 'Hessen': '06', 'Mecklenburg-Vorpommern': '13', 
        'Niedersachsen': '03', 'Nordrhein-Westfalen': '05', 'Rheinland-Pfalz': '07', 
        'Saarland': '10', 'Sachsen': '14', 'Sachsen-Anhalt': '15', 
        'Schleswig-Holstein': '01', 'Thüringen': '16'
    }
    PV_agg['BL_code'] = PV_agg['BL'].map(bl_to_bl_code_mapping)
    
    #about 1 MWp per hectare,meaning 1 MWp per 0.01 km².
    #PV_agg['Total_BRUTTOLEISTUNG'] = PV_agg['Total_BRUTTOLEISTUNG'] / 1000
    PV_agg.rename(columns={'Postleitzahl':'plz_code'}, inplace=True)
    

    blpath='BL-Area_2023.csv' #since it is only for name purposes and without change and a potential point of further work, the year is hardcoded

    BL_Area=pd.read_csv(blpath,sep=';')
    #delete nan values in PV_agg BL_code
    PV_agg.dropna(subset=['BL_code'], inplace=True)
    BL_Area['BL_code'] = BL_Area['BL_code'].astype(int) 
    PV_agg['BL_code'] = PV_agg['BL_code'].astype(int)

    #add every plz_code value from plz_df to PV_agg. lan_code is the value that shoud be put into BL_code. all other values are 0
    
    # Create a new dataframe from plz_df with the columns you need
    new_entries = plz_df[['plz_code', 'lan_code']].copy()

    # Rename the 'lan_code' to 'BL_code' to match the PV_agg DataFrame
    new_entries.rename(columns={'lan_code': 'BL_code'}, inplace=True)

    # Set all other relevant columns to 0
    for col in PV_agg.columns.difference(['plz_code', 'BL_code']):
        new_entries[col] = 0
    new_entries['BL_code'] = new_entries['BL_code'].astype(int)
    new_entries['plz_code'] = new_entries['plz_code'].astype(int)
    
    # Merge the new entries into PV_agg
    # This uses an outer join to ensure all values from both dataframes are included
    #use only the first entry of each plz_code value
    
    PV_agg = pd.merge(PV_agg, new_entries, on='plz_code', how='outer', suffixes=('', '_new'))

    # Where original BL_code is NaN (meaning it was missing), fill it with BL_code_new values
    PV_agg['BL_code'] = PV_agg['BL_code'].fillna(PV_agg['BL_code_new'])

    # Drop the temporary BL_code_new column
    # Correctly selecting multiple columns from a DataFrame
    PV_agg = PV_agg[['plz_code', 'BL', 'Total_BRUTTOLEISTUNG', 'num_installation', 'mean_age','BL_code']]



    # Fill NaN values for other columns in PV_agg with 0, since they might be missing in the new entries
    PV_agg.fillna(0, inplace=True)

    # Optionally convert columns to integer if they must be integer type
    # Convert only 'plz_code' and 'BL_code' columns to integers
    if 'plz_code' in PV_agg.columns:
        PV_agg['plz_code'] = PV_agg['plz_code'].astype(int)
    if 'BL_code' in PV_agg.columns:
        PV_agg['BL_code'] = PV_agg['BL_code'].astype(int)

    #plz_df=plz_df[['plz_code','lan_code']]

    # Create a mapping from BL_code to Bundesland using BL_Area
    bl_code_to_bundesland_mapping = BL_Area.set_index('BL_code')['Bundesland'].to_dict()

    # # Update the 'Bundesland' column in PV_agg using the mapping
    PV_agg['Bundesland'] = PV_agg['BL_code'].map(bl_code_to_bundesland_mapping)
    
    # # Now PV_agg has 'Bundesland' values that match the representations in BL_Area
    # PV_agg drop duplicates
    PV_agg.drop_duplicates(subset='plz_code', keep='first', inplace=True)
    
    BL_Area.to_csv(f'Data/pv_interim/BL-Area_{year_oi}.csv',sep=';', index=False)  
    PV_agg.to_csv(f'Data/pv_interim/PV_agg_{year_oi}_{Typ}.csv',sep=';', index=False)
    return PV_agg

In [21]:
# year=2024
# mastr_pv=pd.read_csv(f'Data/pv_interim/Mastr/2023MASTRFreif.csv', sep=';')
# contraction_df=pd.read_csv(f'Data/pv_interim/contraction/contraction_data_2024.csv', sep=';')
# PV_agg=pd.read_csv(f'Data/pv_interim/PV_agg_2023_Freif.csv', sep=';')
# bonsys=pd.read_csv(f'Data/bonsys2023.csv')
# expansion_target = expansion_df.loc[expansion_df['Year'] == year, 'PV Expansion'].item()/2 #since it's about half and half for Baul and Freif
# quantiles = mastr_pv['Nettonennleistung'].quantile([0.25, 0.5, 0.75]).values #TODO quantiles for PV power

# # Convert plz_code in both dataframes to the same type
#  # Handle possible non-integer or float-formatted 'plz_code' values
#     bonsys['plz_code'] = pd.to_numeric(bonsys['plz_code'], errors='coerce').fillna(0).astype(int)
#     PV_agg['plz_code'] = pd.to_numeric(PV_agg['plz_code'], errors='coerce').fillna(0).astype(int)
#     contraction_df['plz_code'] = pd.to_numeric(contraction_df['plz_code'], errors='coerce').fillna(0).astype(int)
# #add sum of contraction_df Total_Nettonennleistung to expansion_df
# expansion_df.loc[expansion_df['Year'] == year+1, 'PV Expansion'] += contraction_df['Total_Nettonennleistung'].sum()
# # Create a DataFrame of missing 'plz_code' with default 'diff' values

# missing_plz_codes = PV_agg[~PV_agg['plz_code'].isin(bonsys['plz_code'])]['plz_code'].unique()
# missing_plz_df = pd.DataFrame({
#     'plz_code': missing_plz_codes,
#     'building-installations_pv': [0] * len(missing_plz_codes),  # example default values
#     'ground-mounted_installations_pv': [0] * len(missing_plz_codes),
#     'diff': [0] * len(missing_plz_codes)  # or some neutral value
# })
# merged_df = pd.merge(PV_agg, contraction_df, on='plz_code', how='left', suffixes=('', '_contraction'))
# merged_df

In [22]:

# merged_df['Total_Nettonennleistung'].fillna(0, inplace=True)
# merged_df['Anzahl_Anlagen'].fillna(0, inplace=True)
# merged_df['Total_BRUTTOLEISTUNG'] -= merged_df['Total_Nettonennleistung']
# merged_df['num_installation'] -= merged_df['Anzahl_Anlagen']
# # Assumption: New installations have an age of 0 at the time of installation
# merged_df['total_old_age'] = merged_df['num_installation'] * merged_df['mean_age']

# # New installations have an age of 0, so their total age contribution is 0
# # This line is illustrative and can be skipped as it does not change the calculation
# # merged_df['total_new_age'] = merged_df['Anzahl_Anlagen'] * 0

# # Calculate new mean age
# merged_df['new_mean_age'] = (merged_df['total_old_age'] + 0) / (merged_df['num_installation'] + merged_df['Anzahl_Anlagen'])

# # Handle divisions by zero if there are no installations at all
# merged_df['new_mean_age'] = merged_df['new_mean_age'].fillna(0)

# # Optionally, drop the temporary column used for calculation
# #merged_df.drop(columns=['total_old_age'], inplace=True)
# PV_agg=merged_df[['plz_code', 'BL', 'Total_BRUTTOLEISTUNG', 'num_installation', 'mean_age', 'BL_code', 'Bundesland']]


In [61]:
def distribute_pv(year, expansion_df, installation_type):
    # Load datasets
    mastr_pv=pd.read_csv(f'2023MASTR{installation_type}.csv', sep=';')
    contraction_df=pd.read_csv(f'Data/pv_interim/contraction_data_{year}_{installation_type}.csv', sep=';')
    PV_agg=pd.read_csv(f'PV_agg_{year-1}_{installation_type}.csv', sep=';')
    bonsys=pd.read_csv(f'Data/pv_interim/bonsys{year-1}.csv')
    expansion_target = expansion_df.loc[expansion_df['Year'] == year, 'PV Expansion'].item()/2 #since it's about half and half for Baul and Freif
    
    # Convert plz_code in both dataframes to the same type
    # Handle possible non-integer or float-formatted 'plz_code' values
    bonsys['plz_code'] = pd.to_numeric(bonsys['plz_code'], errors='coerce').fillna(0).astype(int)
    PV_agg['plz_code'] = pd.to_numeric(PV_agg['plz_code'], errors='coerce').fillna(0).astype(int)
    contraction_df['plz_code'] = pd.to_numeric(contraction_df['plz_code'], errors='coerce').fillna(0).astype(int)
    #add sum of contraction_df Total_Nettonennleistung to expansion_df
    expansion_df.loc[expansion_df['Year'] == year+1, 'PV Expansion'] += contraction_df['Total_Nettonennleistung'].sum()
    # Create a DataFrame of missing 'plz_code' with default 'diff' values
    expansion_chunk_value=mastr_pv['Nettonennleistung'].quantile(0.75) 
     
    missing_plz_codes = PV_agg[~PV_agg['plz_code'].isin(bonsys['plz_code'])]['plz_code'].unique()
    missing_plz_df = pd.DataFrame({
        'plz_code': missing_plz_codes,
        'building-installations_pv': [0] * len(missing_plz_codes),  # example default values
        'ground-mounted_installations_pv': [0] * len(missing_plz_codes),
        'diff': [0] * len(missing_plz_codes)  # or some neutral value
    })
    merged_df = pd.merge(PV_agg, contraction_df, on='plz_code', how='left', suffixes=('', '_contraction'))
    print(merged_df.columns)
    merged_df['Total_Nettonennleistung'].fillna(0, inplace=True)
    merged_df['Anzahl_Anlagen'].fillna(0, inplace=True)
    merged_df['Total_BRUTTOLEISTUNG'] -= merged_df['Total_Nettonennleistung']
    merged_df['num_installation'] -= merged_df['Anzahl_Anlagen']
    # Assumption: New installations have an age of 0 at the time of installation
    merged_df['total_old_age'] = merged_df['num_installation'] * merged_df['mean_age']

    # New installations have an age of 0, so their total age contribution is 0
    # This line is illustrative and can be skipped as it does not change the calculation
    # merged_df['total_new_age'] = merged_df['Anzahl_Anlagen'] * 0

    # Calculate new mean age
    merged_df['new_mean_age'] = (merged_df['total_old_age'] + 0) / (merged_df['num_installation'] + merged_df['Anzahl_Anlagen'])

    # Handle divisions by zero if there are no installations at all
    merged_df['new_mean_age'] = merged_df['new_mean_age'].fillna(0)

    # Optionally, drop the temporary column used for calculation
    #merged_df.drop(columns=['total_old_age'], inplace=True)
    PV_agg=merged_df[['plz_code', 'BL', 'Total_BRUTTOLEISTUNG', 'num_installation', 'mean_age', 'BL_code', 'Bundesland']]
    #delete duplicates 
    PV_agg.drop_duplicates(subset='plz_code', keep='first', inplace=True)
    bonsys = pd.concat([bonsys, missing_plz_df], ignore_index=True)

    # Calculate the expansion target and determine the total chunks available
    expansion_target = expansion_df.loc[expansion_df['Year'] == year, 'PV Expansion'].item() / 2
    total_chunks = int(expansion_target / expansion_chunk_value)
    print(f"Year: {year}, Installation Type: {installation_type}, Total Chunks Available: {total_chunks}")
    if installation_type == 'Baul':
        base_upBound = 250
    elif installation_type == 'Freif':
        base_upBound = 10

    # Set up the linear programming problem
    prob = pulp.LpProblem("PV_Distribution", pulp.LpMaximize)

    # Define variables for the number of chunks each plz_code can receive
    distribution_units = {}
    for plz_code in PV_agg['plz_code']:
        diff = bonsys[bonsys['plz_code'] == plz_code]['diff'].iloc[0] if not bonsys[bonsys['plz_code'] == plz_code].empty else 0
        if installation_type == 'Baul':
            base_upBound = 250
        elif installation_type == 'Freif':
            base_upBound = 5
        upper_bound = max(0, base_upBound + base_upBound * diff)
        distribution_units[plz_code] = pulp.LpVariable(f"Chunks_{plz_code}", lowBound=0, upBound=upper_bound, cat='Integer')

    # Objective function to maximize the total power distributed
    prob += pulp.lpSum(distribution_units[plz] * expansion_chunk_value for plz in distribution_units)

    # Add constraint to ensure the total power does not exceed the available chunks
    prob += pulp.lpSum(distribution_units[plz] * expansion_chunk_value for plz in distribution_units) <= expansion_target

    # Solve the problem
    prob.solve()

    # Collect and print the results
    if pulp.LpStatus[prob.status] == 'Optimal':
        print("Optimal solution found.")
        for plz in distribution_units:
            units = distribution_units[plz].varValue
            if units > 0:
                distributed_power = units * expansion_chunk_value
                PV_agg.loc[PV_agg['plz_code'] == plz, 'Total_BRUTTOLEISTUNG'] += distributed_power
                PV_agg.loc[PV_agg['plz_code'] == plz, 'num_installation'] += units
                print(f"PLZ {plz}: {units} chunks, {distributed_power} kW added")
    else:
        print("Failed to find an optimal solution.")

    # Update and save data
    PV_agg.to_csv(f'Data/pv_interim/PV_agg_{year}_{installation_type}.csv', sep=';', index=False)
    bonsys.to_csv(f'Data/pv_interim/bonsys{year}.csv', index=False)
    return prob, PV_agg

# # Example usage
# year = 2024
# #expansion_df = pd.DataFrame({'Year': [2024], 'PV Expansion': [20000000]})  # Total expansion for the year
# installation_type = 'Baul'
# #expansion_chunk_value = 2000  # 1 kW per chunk
# prob, PV_agg = distribute_pv(year, expansion_df, installation_type)
# print(PV_agg['Total_BRUTTOLEISTUNG'].sum())

In [20]:
def join_mastrs(PV_agg, PV_aggB, mastr_pv, mastr_pvB,year_oi):
    """
    Joins two pairs of DataFrames: one pair for aggregated PV data and another for detailed MaStR data.

    Parameters:
    - PV_agg: First DataFrame of aggregated PV data.
    - PV_aggB: Second DataFrame of aggregated PV data to be joined with the first.
    - mastr_pv: First DataFrame of detailed MaStR data.
    - mastr_pvB: Second DataFrame of detailed MaStR data to be joined with the first.

    Returns:
    - A tuple of joined DataFrames: (joined PV_agg, joined mastr_pv).
    """
    year_oistr = str(year_oi)
    # Join the two aggregated dataframes using concat for better handling of similar structured data
    joined_PV_agg = pd.concat([PV_agg, PV_aggB], ignore_index=True)

    # Append the second detailed MaStR data to the first
    joined_mastr_pv = pd.concat([mastr_pv, mastr_pvB], ignore_index=True)
    joined_mastr_pv.to_csv(f'Data/pv_interim/mastr_pv_{year_oistr}.csv')
    #save as csv
    joined_PV_agg.to_csv(f'Data/pv_interim/PV_agg_{year_oistr}.csv')
    #joined_mastr_pv.to_csv(f'Data/pv_interim/mastr_pv_{year_oistr}.csv')
    return joined_PV_agg


In [80]:

def convert_irradiance_to_PSH(year, scenario):
    # Create a copy of the dataframe to avoid modifying the original data
    df = pd.read_csv(f'Data/pv_interim/{scenario}/germany_gdf_SDR_{scenario}_{year}.csv')
    
    # Identify all columns with irradiance data for the specified year
    irradiance_columns = [col for col in df.columns if col.startswith(str(year))]

    # Process each column
    for month_col in irradiance_columns:
        # Split to get the month number from the column name, e.g., '2024-01' -> 1
        _, month = map(int, month_col.split('-'))
        # Calculate the number of hours in the month
        hours_in_month = monthrange(year, month)[1] * 24
        
        # Calculate total kWh/m² for the month
        df[f'{month_col}'] = (df[month_col] * hours_in_month) / 1000

    #save the results as a csv file
    df.to_csv(f'Data/pv_interim/PSH_{year}_{scenario}.csv', index=False)
    return df

In [90]:
def calculate_actual_power_output(year_oi, scenario, typ):
    efficiency = 0.23
    aging_factor = 0.007
    area_m2_per_kWp = 6
    muPmax = 0.0034
    T_ref = 25
    year_oistr = str(year_oi)
    PV_agg = pd.read_csv(f'Data/pv_interim/PV_agg_{year_oistr}_{typ}.csv', sep=';')    
    Tpv = pd.read_csv(f'Data/pv_interim/Tpv_df_{scenario}_{year_oistr}.csv', sep=';')
    PSH=pd.read_csv(f'Data/pv_interim/PSH_{year_oistr}_{scenario}.csv')
    daylight_df = pd.read_csv(f'Data/pv_interim/daylight_df_{scenario}_{year_oistr}.csv') 
    # Ensure DataFrame 'plz_code' columns are of the same type for proper merging
    PV_agg['plz_code'] = PV_agg['plz_code'].astype(int)
    Tpv['plz_code'] = Tpv['plz_code'].astype(int)
    PSH['plz_code'] = PSH['plz_code'].astype(int)
    daylight_df['plz_code'] = daylight_df['plz_code'].astype(int)

    # Initialize a list to hold data
    results = []

    # Iterate over each month
    for month in range(1, 13):
        month_str = f"{year_oi}-{str(month).zfill(2)}"
        
        if month_str in Tpv.columns and month_str in PSH.columns and month_str in daylight_df.columns:
            T_cell = Tpv[month_str]
            G = PSH[month_str]
            D = daylight_df[month_str]

            # Calculate the adjusted efficiencies
            eta_age = (1 - aging_factor * PV_agg['mean_age'])
            eta_temp = eta_age * (1 - muPmax * (T_cell - T_ref))

            # Calculate area and power output incorporating daylight hours fraction
            A = PV_agg['Total_BRUTTOLEISTUNG']
            power_output = (G * A * eta_temp*0.87 ).clip(lower=0) #13% wechslerverluste

            # Append each row to results
            for plz_code, production in zip(PV_agg['plz_code'], power_output):
                results.append({
                    'plz_code': plz_code,
                    'year': year_oi,
                    'month': month,
                    'Production': production
                })

    # Convert list to DataFrame
    results_df = pd.DataFrame(results)
 
    PVsum = results_df['Production'].sum() / 1000000000
    print(f'Production in {year_oistr} is {PVsum} TWh for {typ} installations')
    # Save to CSV
    results_df.to_csv(f'Data/Results/PV_power_output_{scenario}_{year_oi}_{typ}.csv', index=False)
    
    return results_df




In [92]:
def merge_Power_outputs(year, scenario):
    ProdFreif = pd.read_csv(f'Data/Results/PV_power_output_{scenario}_{year}_Freif.csv', sep=',')
    ProdBaul = pd.read_csv(f'Data/Results/PV_power_output_{scenario}_{year}_Baul.csv', sep=',') 
    
    
    # Merging the dataframes on 'plz_code', 'year', and 'month' with an outer join to keep all plz_code values
    merged_df = pd.merge(ProdFreif, ProdBaul, on=['plz_code', 'year', 'month'], how='outer', suffixes=('_ProdFreif', '_ProdBaul'))

    # Filling NaN values with 0 for addition
    merged_df['Production_ProdFreif'].fillna(0, inplace=True)
    merged_df['Production_ProdBaul'].fillna(0, inplace=True)

    # Adding the Production values from both dataframes
    merged_df['Production'] = merged_df['Production_ProdFreif'] + merged_df['Production_ProdBaul']

    # Dropping the individual production columns from each dataframe
    merged_df.drop(columns=['Production_ProdFreif', 'Production_ProdBaul'], inplace=True)
    print(f'{merged_df["Production"].sum()/1000000000} TWh')
    



    merged_df.to_csv(f'Data/Results/PV_power_output_{scenario}_{year}.csv', sep=',', index=False)

In [14]:

# plz_shapefile=gpd.read_file('shp/georef-germany-postleitzahl/georef-germany-postleitzahl.shp')
# plz_shapefile=plz_shapefile.to_crs('EPSG:4326')
# sdr=xr.open_dataset('get_data/PV-Input/rsds_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_SMHI-RCA4_v1a_mon_202101-203012.nc') 
# ws=xr.open_dataset('get_data/PV-Input/sfcWind_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_SMHI-RCA4_v1a_mon_202101-203012.nc')
# at=xr.open_dataset('get_data/PV-Input/tasmax_EUR-11_MPI-M-MPI-ESM-LR_rcp26_r1i1p1_SMHI-RCA4_v1a_mon_202101-203012.nc')
# year_oi=2024



import xarray as xr
import datetime
start_year = 2023
end_year = 2023
# Define the base path and scenarios
base_path = "Data/"
scenarios = ['RCP26', 'RCP45', 'RCP85']
variables = {
    'sdr': f'rsds_EUR-11_MPI-M-MPI-ESM-LR_{scenario}_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc',
    'ws': f'sfcWind_EUR-11_MPI-M-MPI-ESM-LR_{scenario}_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc',
    'at': f'tasmax_EUR-11_MPI-M-MPI-ESM-LR_{scenario}_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc'
}


for scenario in scenarios:
    print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]: Processing for scenario {scenario}")
    #for year in range(start_year, end_year + 1):
    sdr=xr.open_dataset(variables['sdr'].format(scenario=scenario))
    ws=xr.open_dataset(variables['ws'].format(scenario=scenario))
    at=xr.open_dataset(variables['at'].format(scenario=scenario))

    germany_gdfSDR = create_germany_gdf_from_ds(sdr, 'rsds')
    germany_gdfWS = create_germany_gdf_from_ds(ws, 'sfcWind')
    germany_gdfAT = create_germany_gdf_from_ds(at, 'tasmax')

    germany_gdfSDR=process_year(germany_gdfSDR, plz_shapefile, 'SDR', scenario)
    mapping_dictSDR=generate_id_to_plz_mapping(germany_gdfSDR)   
    
    germany_gdfWS=process_year(germany_gdfWS,plz_shapefile, 'WS', scenario)
    mapping_dictWS=generate_id_to_plz_mapping(germany_gdfWS)   

    germany_gdfAT=process_year (germany_gdfAT,plz_shapefile, 'AT', scenario)
    mapping_dictAT=generate_id_to_plz_mapping(germany_gdfAT)   



[2024-05-30 18:02:28]: Processing for scenario RCP26


KeyboardInterrupt: 

In [94]:
start_year = 2023
end_year = 2030
calculate_annual_contraction(start_year, end_year,'Freif', input_dir='Data/pv_interim', output_dir='Data/pv_interim/')
calculate_annual_contraction(start_year, end_year,'Baul', input_dir='Data/pv_interim', output_dir='Data/pv_interim/')

mastr_pv, expansion_df=filter_active_installations('Freif',2024, expansion_df) #passt
mastr_pvB,expansion_df=filter_active_installations('Baul',2024,expansion_df)

PV_agg=aggregate_on_PLZ_level('Freif', start_year,plz_df)
PV_aggB=aggregate_on_PLZ_level('Baul', start_year,plz_df)


Contraction data for year 2023 saved, focusing on installations exactly 25 years old.
Contraction data for year 2024 saved, focusing on installations exactly 25 years old.
Contraction data for year 2025 saved, focusing on installations exactly 25 years old.
Contraction data for year 2026 saved, focusing on installations exactly 25 years old.
Contraction data for year 2027 saved, focusing on installations exactly 25 years old.
Contraction data for year 2028 saved, focusing on installations exactly 25 years old.
Contraction data for year 2029 saved, focusing on installations exactly 25 years old.
Contraction data for year 2030 saved, focusing on installations exactly 25 years old.


C:\Users\klarw\AppData\Local\Temp\ipykernel_6512\1747662707.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  mastr_data = pd.read_csv(file_path, sep=';')


Contraction data for year 2023 saved, focusing on installations exactly 25 years old.
Contraction data for year 2024 saved, focusing on installations exactly 25 years old.
Contraction data for year 2025 saved, focusing on installations exactly 25 years old.
Contraction data for year 2026 saved, focusing on installations exactly 25 years old.
Contraction data for year 2027 saved, focusing on installations exactly 25 years old.
Contraction data for year 2028 saved, focusing on installations exactly 25 years old.
Contraction data for year 2029 saved, focusing on installations exactly 25 years old.
Contraction data for year 2030 saved, focusing on installations exactly 25 years old.
Number of deprecated installations in 2024: 77
Power of deprecated installations in 2024: 1075.0619999999997 kWp


C:\Users\klarw\AppData\Local\Temp\ipykernel_6512\1366450366.py:18: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  mastr_pv = pd.read_csv(f'{year_befoistr}MASTR{Typ}.csv', sep=';')


KeyboardInterrupt: 

In [83]:

import datetime
from tqdm import tqdm
# Inner loop for each year in the specified range
# mastr_pv, expansion_df=filter_active_installations('Freif',2024, expansion_df) #passt
# mastr_pvB,expansion_df=filter_active_installations('Baul',2024,expansion_df) #passt
# print('filtering fine')
# PV_agg=aggregate_on_PLZ_level('Freif', 2024,plz_df) #passt
# PV_aggB=aggregate_on_PLZ_level('Baul', 2024,plz_df) #passt
start_year = 2023
end_year = 2023
for year in range(start_year, end_year + 1):
    print(f"Processing year: {year}")
    year_oi=year

    prob, PV_agg=distribute_pv( year,expansion_df,'Freif')
    print(f"[{datetime.datetime.now()}]: PV_agg and mastr_pv distributed for year {year_oi}. Sum of power Freifläche: {PV_agg['Total_BRUTTOLEISTUNG'].sum()} kWp") 

    prob, PV_aggB=distribute_pv( year,expansion_df,'Baul')
    print(f"[{datetime.datetime.now()}]: PV_agg and mastr_pv distributed for year {year_oi}. Sum of power Baulich: {PV_aggB['Total_BRUTTOLEISTUNG'].sum()} kWp") 

    PV_agg=join_mastrs(PV_agg, PV_aggB, mastr_pv, mastr_pvB,year_oi)#passt
    print(f"[{datetime.datetime.now()}]: PV_agg and mastr_pv joined for year {year_oi} ")


    germany_gdfSDR = create_germany_gdf_from_ds(sdr, 'rsds')
    germany_gdfWS = create_germany_gdf_from_ds(ws, 'sfcWind')
    germany_gdfAT = create_germany_gdf_from_ds(at, 'tasmax')
    print(f"[{datetime.datetime.now()}]: Germany GeoDataFrames created for year {year_oi}")
    
    
    # germany_gdfSDR=save_years(germany_gdfSDR, 'rsds')
    # germany_gdfWS=save_years(germany_gdfWS, 'sfcWind')
    # germany_gdfAT=save_years(germany_gdfAT, 'tas')
    
    
    germany_gdfSDR=map_data_to_plz(germany_gdfWS, mapping_dictSDR, 'SDR', year_oi)
    germany_gdfWS=map_data_to_plz(germany_gdfWS, mapping_dictWS,'WS', year_oi)
    germany_gdfAT=map_data_to_plz(germany_gdfAT, mapping_dictAT,'AT', year_oi)


    
    print(f'Final{year} done at [{datetime.datetime.now()}]')


# Define the base path and scenarios
# base_path = "get_data/PV-Input/"
# scenarios = ['rcp26', 'rcp45', 'rcp85']
# variables = {
#     'sdr': 'rsds_EUR-11_MPI-M-MPI-ESM-LR_{scenario}_r1i1p1_SMHI-RCA4_v1a_mon_202101-205012.nc',
#     'ws': 'sfcWind_EUR-11_MPI-M-MPI-ESM-LR_{scenario}_r1i1p1_SMHI-RCA4_v1a_mon_202101-20501212.nc',
#     'at': 'tasmax_EUR-11_MPI-M-MPI-ESM-LR_{scenario}_r1i1p1_SMHI-RCA4_v1a_mon_202101-20501212.nc'
# }







Processing year: 2023
Index(['plz_code', 'BL', 'Total_BRUTTOLEISTUNG', 'num_installation',
       'mean_age', 'BL_code', 'Bundesland', 'Total_Nettonennleistung',
       'Anzahl_Anlagen', 'Bundesland_contraction'],
      dtype='object')
Year: 2023, Installation Type: Freif, Total Chunks Available: 0


C:\Users\klarw\AppData\Local\Temp\ipykernel_6512\1789911382.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Total_Nettonennleistung'].fillna(0, inplace=True)
C:\Users\klarw\AppData\Local\Temp\ipykernel_6512\1789911382.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

Optimal solution found.
[2024-05-30 18:58:04.716668]: PV_agg and mastr_pv distributed for year 2023. Sum of power Freifläche: 14980321.456 kWp


C:\Users\klarw\AppData\Local\Temp\ipykernel_6512\1789911382.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  mastr_pv=pd.read_csv(f'2023MASTR{installation_type}.csv', sep=';')


Index(['plz_code', 'BL', 'Total_BRUTTOLEISTUNG', 'num_installation',
       'mean_age', 'BL_code', 'Bundesland', 'Total_Nettonennleistung',
       'Anzahl_Anlagen', 'Bundesland_contraction'],
      dtype='object')
Year: 2023, Installation Type: Baul, Total Chunks Available: 0


C:\Users\klarw\AppData\Local\Temp\ipykernel_6512\1789911382.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Total_Nettonennleistung'].fillna(0, inplace=True)
C:\Users\klarw\AppData\Local\Temp\ipykernel_6512\1789911382.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

KeyboardInterrupt: 

In [ ]:
# for scenario in scenarios:
#     germany_gdfAT=pd.read_csv(f'Data/pv_interim/Germany_gdf_AT_{scenario}.csv')
#     germany_gdfAT=save_yearly_csv(germany_gdfAT, 2023, 2050, scenario,'germany_gdfAT')
#     germany_gdfWS=pd.read_csv(f'Data/pv_interim/Germany_gdf_WS_{scenario}.csv')
#     germany_gdfWS=save_yearly_csv(germany_gdfWS, 2023, 2050, scenario,'germany_gdfWS')
#     germany_gdfSDR=pd.read_csv(f'Data/pv_interim/Germany_gdf_SDR_{scenario}.csv')
#     germany_gdfSDR=save_yearly_csv(germany_gdfSDR, 2023, 2050, scenario,'germany_gdfSDR')
    

In [ ]:
def check_plz_code_matches(df1, df2, key='plz_code'):
    """
    Check for matching and mismatching PLZ codes between two dataframes.

    Parameters:
    - df1 (DataFrame): First dataframe to compare.
    - df2 (DataFrame): Second dataframe to compare.
    - key (str): The column name to compare, default is 'plz_code'.

    Returns:
    - dict: A dictionary containing sets of matching and mismatching PLZ codes.
    """
    # Convert PLZ codes to integer to ensure comparison accuracy
    df1[key] = df1[key].astype(int)
    df2[key] = df2[key].astype(int)

    # Create sets of PLZ codes from both dataframes
    plz_set_df1 = set(df1[key])
    plz_set_df2 = set(df2[key])

    # Find matches and mismatches
    matches = plz_set_df1.intersection(plz_set_df2)
    only_in_df1 = plz_set_df1 - plz_set_df2
    only_in_df2 = plz_set_df2 - plz_set_df1

    # Output the results
    print(f"Matching PLZ codes between the two dataframes: {len(matches)}")
    print(f"PLZ codes only in the first dataframe: {len(only_in_df1)}")
    print(f"PLZ codes only in the second dataframe: {len(only_in_df2)}")

    return {
        'matches': matches,
        'only_in_df1': only_in_df1,
        'only_in_df2': only_in_df2
    }

# Example usage
result = check_plz_code_matches(bonsys, PV_agg)
print("Matches:", result['matches'])
print("Only in bonsys:", result['only_in_df1'])
print("Only in PV_agg:", result['only_in_df2'])


In [97]:
import pandas as pd
import xarray as xr
import datetime

# Define the base path and scenarios
base_path = ""
start_year = 2023
end_year = 2031
variables = {
    'sdr': 'rsds_EUR-11_MPI-M-MPI-ESM-LR_{scenario}_r1i1p1_SMHI-RCA4_v1a_mon_{start_year}01-{end_year}12.nc',
    'ws': 'sfcWind_EUR-11_MPI-M-MPI-ESM-LR_{scenario}_r1i1p1_SMHI-RCA4_v1a_mon_{start_year}01-{end_year}12.nc',
    'at': 'tasmax_EUR-11_MPI-M-MPI-ESM-LR_{scenario}_r1i1p1_SMHI-RCA4_v1a_mon_{start_year}01-{end_year}12.nc'
}


scenarios = ['RCP26','RCP45', 'RCP85']
# # Outermost loop for each RCP scenario
# for scenario in scenarios:
#     print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]: Processing for scenario {scenario}")

#     # Inner loop for each year in the specified range
#     for year in range(start_year, end_year + 1):
#         year_oistr=str(year) 
#         convert_irradiance_to_PSH( year, scenario)

# Outermost loop for each RCP scenario
for scenario in scenarios:
    print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}]: Processing for scenario {scenario}")

    # Inner loop for each year in the specified range
    for year in range(start_year, end_year + 1):
        year_oistr=str(year)
        avg_daily_sunhours(year, scenario)
        calculate_Tpv(year,scenario) 
        convert_irradiance_to_PSH( year, scenario)
        Pout=calculate_actual_power_output(year_oistr,scenario, 'Freif')
        PoutB=calculate_actual_power_output(year_oistr,scenario, 'Baul')
        merge_Power_outputs(year,scenario)

[2024-05-30 19:26:45]: Processing for scenario RCP26
Production in 2023 is 17.486456505305775 TWh for Freif installations
Production in 2023 is 44.13703123448 TWh for Baul installations


C:\Users\klarw\AppData\Local\Temp\ipykernel_6512\3722009210.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Production_ProdFreif'].fillna(0, inplace=True)
C:\Users\klarw\AppData\Local\Temp\ipykernel_6512\3722009210.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

61.623487739785766 TWh


FileNotFoundError: [Errno 2] No such file or directory: 'Data/pv_interim/RCP26/germany_gdf_SDR_RCP26_2024.csv'